In [1]:
# 当y、yhat、ycomp是从所有单品中取值组成的序列时，当在第一大类中出现（y-ycomp==0），不能直接区分没有补偿和完全准确补偿的情况；
# 当在第二大类中出现（y-yhat==0），不能直接区分没有预测和完全准确预测的情况；
# 当在第三大类中出现（y-yhat==y-ycomp==0），不能直接区分没有预测、没有补偿、完全预测、完全补偿这四种情况；
# 基于上述原因，会导致无法准确统计某些分项的指标；建议y、yhat、ycomp从所有发生补偿的单品中取值，将没有发生补偿的单品排除。

# import random
# import pandas as pd
# import numpy as np

# weights = []
# for i in range(-99,1):
#     weights.append((0.94)**i)

# k1, k2, k3 = 5000, 4000, 1100
# y = pd.Series(random.choices(range(0,100), k=k1, weights=weights))  # 生成随机序列，模拟k1个单品在某一周的真实销量

# z1 = pd.Series(10*np.sin(random.choices(range(0,5000), k=k2)))  # 假定k1个单品中只有k2个有预测销量
# z2 = pd.Series([0 for _ in range(k1-k2)])
# z3 = pd.concat([z1, z2])
# z4 = z3.sample(frac=1, replace=True)  # 打乱series的顺序
# z4.reset_index(drop=True, inplace=True)
# yhat = y + z4

# z5 = pd.Series(10*np.sin(random.choices(range(0,5000), k=k3)))  # 假定k1个单品中只有k3个发生补偿
# z6 = pd.Series([0 for _ in range(k1-k3)])
# z7 = pd.concat([z1, z2])
# z8 = z7.sample(frac=1, replace=True)  # 打乱series的顺序
# z8.reset_index(drop=True, inplace=True)
# ycomp = y + z8

In [2]:
import random
import pandas as pd
import numpy as np

weights = []
for i in range(-99,1):
    weights.append((0.94)**i)

#  真实值组成的序列y、预测值组成的序列yhat、补偿值组成的序列ycomp，这三条序列组成中的单品是一一对应的，即同一单品在三条序列中的index相同
k1 = 5000
y = pd.Series(random.choices(range(0,100), k=k1, weights=weights))  # 生成随机序列，模拟5000个单品在某一周的真实销量

z1 = pd.Series(np.sin(random.choices(range(0,100), k=k1)))  # 假定这5000个是都有预测销量的单品
yhat = y + z1

z2 = pd.Series(np.sin(random.choices(range(0,100), k=k1)))  # 并且假定这5000个是都发生补偿的单品
ycomp = y + z2

In [3]:
# 得到经过第一层的三大类条件判断之后的真实值序列y_s1/2/3、预测值序列yhat_s1/2/3、补偿值序列ycomp_s1/2/3.

# 1. 补偿变好这一大类：
y_s1 = y[abs(y-yhat)>abs(y-ycomp)]  # 选出y序列中补偿变好的这一大类单品
yhat_s1 = yhat[abs(y-yhat)>abs(y-ycomp)]  # 选出yhat序列中补偿变好的这一大类单品
ycomp_s1 = ycomp[abs(y-yhat)>abs(y-ycomp)]  # 选出ycomp序列中补偿变好的这一大类单品

# 2. 补偿变差这一大类：
y_s2 = y[abs(y-yhat)<abs(y-ycomp)]  # 选出y序列中补偿变差的这一大类单品
yhat_s2 = yhat[abs(y-yhat)<abs(y-ycomp)]  # 选出yhat序列中补偿变差的这一大类单品
ycomp_s2 = ycomp[abs(y-yhat)<abs(y-ycomp)]  # 选出ycomp序列中补偿变差的这一大类单品

# 3. 补偿无效果这一大类：
y_s3 = y[abs(y-yhat)==abs(y-ycomp)]  # 选出y序列中补偿无效果的这一大类单品
yhat_s3 = yhat[abs(y-yhat)==abs(y-ycomp)]  # 选出yhat序列中补偿无效果的这一大类单品
ycomp_s3 = ycomp[abs(y-yhat)==abs(y-ycomp)]  # 选出ycomp序列中补偿无效果的这一大类单品

In [73]:
if len(y) == len(yhat) == len(ycomp):  # 若该条件满足，则必有：len(y_s1)==len(yhat_s1)==len(ycomp_s1), len(y_s2)==len(yhat_s2)==len(ycomp_s2), len(y_s3)==len(yhat_s3)==len(ycomp_s3).
    # 因为这里只需获取三大类情况各自的单品数量m1/m2/m3，所以从y、yhat、ycomp任一序列中选取单品，得到的m1/m2/m3都相等。下面是从y序列中选取单品，得到的yhat序列的长度m1/m2/m3.
    m1, m2, m3 = len(y_s1), len(y_s2), len(y_s3)
    mt = m1+m2+m3
    if mt == k1:
        print('计算正确，补偿变好、变坏、无效果三种情况的单品数相加与发生补偿的单品总数相等：\n')
        print('补偿变好的单品数有：{:.0f}个，占总单品比例：{:.2f}%'.format(m1, m1/mt*100))
        print('补偿变差的单品数有：{:.0f}个，占总单品比例：{:.2f}%'.format(m2, m2/mt*100))
        print('补偿无效果的单品数有：{:.0f}个，占总单品比例：{:.2f}%'.format(m3, m3/mt*100))
    else: raise Exception('计算有误，变好、变化、无效果三种情况的单品数相加与发生补偿的单品总数不等。')
else: raise Exception('真实值、预测值、补偿值组成的序列长度不等，建议化为等长，以避免后续某些指标代表的成分不唯一的情况出现。')


计算正确，补偿变好、变坏、无效果三种情况的单品数相加与发生补偿的单品总数相等：

补偿变好的单品数有：2465个，占总单品比例：49.30%
补偿变差的单品数有：2476个，占总单品比例：49.52%
补偿无效果的单品数有：59个，占总单品比例：1.18%


In [5]:
# 在经过第一层条件判断之后，得到经过第二层条件判断的各个真实值序列、预测值序列、补偿值序列


# 对于补偿变好这一大类中的6小类情况：

# 1.1 预测补偿均低，补偿同侧变好：
y_s1_c1 = y_s1[(y_s1-yhat_s1>0)&(y_s1-ycomp_s1>0)]
yhat_s1_c1 = yhat_s1[(y_s1-yhat_s1>0)&(y_s1-ycomp_s1>0)]
ycomp_s1_c1 = ycomp_s1[(y_s1-yhat_s1>0)&(y_s1-ycomp_s1>0)]
# 1.2 预测补偿均高，补偿同侧变好：
y_s1_c2 = y_s1[(y_s1-yhat_s1<0)&(y_s1-ycomp_s1<0)]
yhat_s1_c2 = yhat_s1[(y_s1-yhat_s1<0)&(y_s1-ycomp_s1<0)]
ycomp_s1_c2 = ycomp_s1[(y_s1-yhat_s1<0)&(y_s1-ycomp_s1<0)]
# 1.3 预低补高，补偿对侧变好：
y_s1_c3 = y_s1[(y_s1-yhat_s1>0)&(y_s1-ycomp_s1<0)]
yhat_s1_c3 = yhat_s1[(y_s1-yhat_s1>0)&(y_s1-ycomp_s1<0)]
ycomp_s1_c3 = ycomp_s1[(y_s1-yhat_s1>0)&(y_s1-ycomp_s1<0)]
# 1.4 预高补低，补偿对侧变好：
y_s1_c4 = y_s1[(y_s1-yhat_s1<0)&(y_s1-ycomp_s1>0)]
yhat_s1_c4 = yhat_s1[(y_s1-yhat_s1<0)&(y_s1-ycomp_s1>0)]
ycomp_s1_c4 = ycomp_s1[(y_s1-yhat_s1<0)&(y_s1-ycomp_s1>0)]
# 1.5 预低，完全补偿：
y_s1_c5 = y_s1[(y_s1-yhat_s1>0)&(y_s1-ycomp_s1==0)]
yhat_s1_c5 = yhat_s1[(y_s1-yhat_s1>0)&(y_s1-ycomp_s1==0)]
ycomp_s1_c5 = ycomp_s1[(y_s1-yhat_s1>0)&(y_s1-ycomp_s1==0)]
# 1.6 预高，完全补偿：
y_s1_c6 = y_s1[(y_s1-yhat_s1<0)&(y_s1-ycomp_s1==0)]
yhat_s1_c6 = yhat_s1[(y_s1-yhat_s1<0)&(y_s1-ycomp_s1==0)]
ycomp_s1_c6 = ycomp_s1[(y_s1-yhat_s1<0)&(y_s1-ycomp_s1==0)]


# 对于补偿变差这一大类中的6小类情况：

# 2.1 预测补偿均低，补偿同侧变差：
y_s2_c1 = y_s2[(y_s2-yhat_s2>0)&(y_s2-ycomp_s2>0)]
yhat_s2_c1 = yhat_s2[(y_s2-yhat_s2>0)&(y_s2-ycomp_s2>0)]
ycomp_s2_c1 = ycomp_s2[(y_s2-yhat_s2>0)&(y_s2-ycomp_s2>0)]
# 2.2 预测补偿均高，补偿同侧变差：
y_s2_c2 = y_s2[(y_s2-yhat_s2<0)&(y_s2-ycomp_s2<0)]
yhat_s2_c2 = yhat_s2[(y_s2-yhat_s2<0)&(y_s2-ycomp_s2<0)]
ycomp_s2_c2 = ycomp_s2[(y_s2-yhat_s2<0)&(y_s2-ycomp_s2<0)]
# 2.3 预低补高，补偿对侧变差：
y_s2_c3 = y_s2[(y_s2-yhat_s2>0)&(y_s2-ycomp_s2<0)]
yhat_s2_c3 = yhat_s2[(y_s2-yhat_s2>0)&(y_s2-ycomp_s2<0)]
ycomp_s2_c3 = ycomp_s2[(y_s2-yhat_s2>0)&(y_s2-ycomp_s2<0)]
# 2.4 预高补低，补偿对侧变差：
y_s2_c4 = y_s2[(y_s2-yhat_s2<0)&(y_s2-ycomp_s2>0)]
yhat_s2_c4 = yhat_s2[(y_s2-yhat_s2<0)&(y_s2-ycomp_s2>0)]
ycomp_s2_c4 = ycomp_s2[(y_s2-yhat_s2<0)&(y_s2-ycomp_s2>0)]
# 2.5 完全准确预测，补偿偏低：
y_s2_c5 = y_s2[(y_s2-yhat_s2==0)&(y_s2-ycomp_s2>0)]
yhat_s2_c5 = yhat_s2[(y_s2-yhat_s2==0)&(y_s2-ycomp_s2>0)]
ycomp_s2_c5 = ycomp_s2[(y_s2-yhat_s2==0)&(y_s2-ycomp_s2>0)]
# 2.6 完全准确预测，补偿偏高：
y_s2_c6 = y_s2[(y_s2-yhat_s2==0)&(y_s2-ycomp_s2<0)]
yhat_s2_c6 = yhat_s2[(y_s2-yhat_s2==0)&(y_s2-ycomp_s2<0)]
ycomp_s2_c6 = ycomp_s2[(y_s2-yhat_s2==0)&(y_s2-ycomp_s2<0)]


# 对于补偿无效果这一大类中的几小类情况：



In [6]:
# 1. 对于第一大类即补偿变好的情况

m11, m12 = len(y_s1_c1), len(y_s1_c2)  # 同侧补偿且变好
m13, m14 = len(y_s1_c3), len(y_s1_c4)  # 对侧补偿且变好
m15, m16 = len(y_s1_c5), len(y_s1_c6)  # 完全补偿
m1syn, m1opp, m1com = m11+m12, m13+m14, m15+m16  # 补偿变好时同侧、对侧、完全补偿的单品数
if m11+m12+m13+m14+m15+m16 == m1:
    print('计算正确，当补偿变好时，其下6种子情况的单品总数相加与补偿变好的单品总数相等：\n')
    print('预测销量与补偿后销量均比真实销量低，补偿变好且发生在同侧，该种情况有：{:.0f}个单品，\
占补偿变好单品数的比例为：{:.2f}%'.format(m11, m11/m1*100))
    print('预测销量与补偿后销量均比真实销量高，补偿变好且发生在同侧，该种情况有：{:.0f}个单品，\
占补偿变好单品数的比例为：{:.2f}%'.format(m12, m12/m1*100))
    print('预测销量偏低，补偿后销量偏高，补偿变好且发生在对侧，该种情况有：{:.0f}个单品，\
占补偿变好单品数的比例为：{:.2f}%'.format(m13, m13/m1*100))
    print('预测销量偏高，补偿后销量偏低，补偿变好且发生在对侧，该种情况有：{:.0f}个单品，\
占补偿变好单品数的比例为：{:.2f}%'.format(m14, m14/m1*100))
    print('预测销量偏低，完全补偿，该种情况有：{:.0f}个单品，占补偿变好单品数的比例为：{:.2f}%'.format(m15, m15/m1*100))
    print('预测销量偏高，完全补偿，该种情况有：{:.0f}个单品，占补偿变好单品数的比例为：{:.2f}%'.format(m16, m16/m1*100))
    print('补偿变好且发生在同侧的单品数有：{:.0f}个，占补偿变好单品数的比例为：{:.2f}%'.format(m1syn, m1syn/m1*100))
    print('补偿变好且发生在对侧的单品数有：{:.0f}个，占补偿变好单品数的比例为：{:.2f}%'.format(m1opp, m1opp/m1*100))
    print('完全补偿的单品数有：{:.0f}个，占补偿变好单品数的比例为：{:.2f}%'.format(m1com, m1com/m1*100))  
else: raise Exception('计算有误，当补偿变好时，其下6种子情况的单品总数相加与补偿变好的单品总数不等。')

计算正确，当补偿变好时，其下6种子情况的单品总数相加与补偿变好的单品总数相等：

预测销量与补偿后销量均比真实销量低，补偿变好且发生在同侧，该种情况有：592个单品，占补偿变好单品数的比例为：24.02%
预测销量与补偿后销量均比真实销量高，补偿变好且发生在同侧，该种情况有：609个单品，占补偿变好单品数的比例为：24.71%
预测销量偏低，补偿后销量偏高，补偿变好且发生在对侧，该种情况有：616个单品，占补偿变好单品数的比例为：24.99%
预测销量偏高，补偿后销量偏低，补偿变好且发生在对侧，该种情况有：599个单品，占补偿变好单品数的比例为：24.30%
预测销量偏低，完全补偿，该种情况有：26个单品，占补偿变好单品数的比例为：1.05%
预测销量偏高，完全补偿，该种情况有：23个单品，占补偿变好单品数的比例为：0.93%
补偿变好且发生在同侧的单品数有：1201个，占补偿变好单品数的比例为：48.72%
补偿变好且发生在对侧的单品数有：1215个，占补偿变好单品数的比例为：49.29%
完全补偿的单品数有：49个，占补偿变好单品数的比例为：1.99%


In [11]:
# 1.1 预测补偿均低，补偿同侧变好：
print('预测补偿均低，补偿同侧变好时：\n')

pva_s1_c1 = abs(y_s1_c1 - yhat_s1_c1)  # 预测偏差量
pva_s1_c1_sts = pva_s1_c1.describe()
pvr_s1_c1 = abs(pva_s1_c1[y_s1_c1!=0] / y_s1_c1[y_s1_c1!=0])*100  # 预测偏差量与真实销量百分比，在此时的条件下真实值(分母)为0的单品不能参与该指标计算，否则会使下面describe的统计指标失真。所以pvr_s1_c1计算的是真实销量不为0的那些单品。
pvr_s1_c1_sts = pvr_s1_c1.describe()
print('预测偏差量的统计指标：')
print(pva_s1_c1_sts)
print('预测偏差量与真实销量百分比的统计指标：')
print(pvr_s1_c1_sts)
print('----------------------------------------')

cva_s1_c1 = abs(y_s1_c1 - ycomp_s1_c1)  # 补偿后剩余偏差量
cva_s1_c1_sts = cva_s1_c1.describe()
cvr_s1_c1 = abs(cva_s1_c1[y_s1_c1!=0] / y_s1_c1[y_s1_c1!=0])*100  # 补偿后剩余偏差量与真实销量百分比，在此时的条件下真实值(分母)为0的单品不能参与该指标计算，否则会使下面describe的统计指标失真。所以cvr_s1_c1计算的是真实销量不为0的那些单品。
cvr_s1_c1_sts = cvr_s1_c1.describe()
print('补偿后剩余偏差量的统计指标：')
print(cva_s1_c1_sts)
print('补偿后剩余偏差量与真实销量百分比的统计指标：')
print(cvr_s1_c1_sts)
print('----------------------------------------')

ca_s1_c1 = abs(yhat_s1_c1 - ycomp_s1_c1)  # 补偿量
ca_s1_c1_sts = ca_s1_c1.describe()
car_s1_c1 = abs(ca_s1_c1[pva_s1_c1!=0] / pva_s1_c1[pva_s1_c1!=0])*100  # 补偿量占预测偏差量百分比
car_s1_c1_sts = car_s1_c1.describe()
print('补偿量的统计指标：')
print(ca_s1_c1_sts)
print('补偿量占预测偏差量百分比的统计指标：')
print(car_s1_c1_sts)


预测补偿均低，补偿同侧变好时：

预测偏差量的统计指标：
count    592.000000
mean       0.838312
std        0.205491
min        0.114785
25%        0.756802
50%        0.923458
75%        0.988032
max        0.999990
dtype: float64
预测偏差量与真实销量百分比的统计指标：
count    565.000000
mean      14.041330
std       19.897134
min        0.352495
25%        3.160419
50%        6.636339
75%       15.982071
max       99.999021
dtype: float64
----------------------------------------
补偿后剩余偏差量的统计指标：
count    592.000000
mean       0.459126
std        0.286215
min        0.008851
25%        0.245252
50%        0.428183
75%        0.676772
max        0.999207
dtype: float64
补偿后剩余偏差量与真实销量百分比的统计指标：
count    565.000000
mean       7.792630
std       13.827291
min        0.013211
25%        1.303347
50%        3.048106
75%        7.568025
max       95.375265
dtype: float64
----------------------------------------
补偿量的统计指标：
count    592.000000
mean       0.379186
std        0.264777
min        0.000548
25%        0.147668
50%        0.338865
7

In [10]:
# 1.2 预测补偿均高，补偿同侧变好：
print('预测补偿均高，补偿同侧变好时：\n')

pva_s1_c2 = abs(y_s1_c2 - yhat_s1_c2)  # 预测偏差量
pva_s1_c2_sts = pva_s1_c2.describe()
pvr_s1_c2 = abs(pva_s1_c2[y_s1_c2!=0] / y_s1_c2[y_s1_c2!=0])*100  # 预测偏差量与真实销量百分比
pvr_s1_c2_sts = pvr_s1_c2.describe()
print('预测偏差量的统计指标：')
print(pva_s1_c2_sts)
print('预测偏差量与真实销量百分比的统计指标：')
print(pvr_s1_c2_sts)
print('----------------------------------------')

cva_s1_c2 = abs(y_s1_c2 - ycomp_s1_c2)  # 补偿后剩余偏差量
cva_s1_c2_sts = cva_s1_c2.describe()
cvr_s1_c2 = abs(cva_s1_c2[y_s1_c2!=0] / y_s1_c2[y_s1_c2!=0])*100  # 补偿后剩余偏差量与真实销量百分比
cvr_s1_c2_sts = cvr_s1_c2.describe()
print('补偿后剩余偏差量的统计指标：')
print(cva_s1_c2_sts)
print('补偿后剩余偏差量与真实销量百分比的统计指标：')
print(cvr_s1_c2_sts)
print('----------------------------------------')

ca_s1_c2 = abs(yhat_s1_c2 - ycomp_s1_c2)  # 补偿量
ca_s1_c2_sts = ca_s1_c2.describe()
car_s1_c2 = abs(ca_s1_c2[pva_s1_c2!=0] / pva_s1_c2[pva_s1_c2!=0])*100  # 补偿量占预测偏差量百分比
car_s1_c2_sts = car_s1_c2.describe()
print('补偿量的统计指标：')
print(ca_s1_c2_sts)
print('补偿量占预测偏差量百分比的统计指标：')
print(car_s1_c2_sts)

预测补偿均高，补偿同侧变好时：

预测偏差量的统计指标：
count    609.000000
mean       0.824932
std        0.198863
min        0.123573
25%        0.733190
50%        0.909297
75%        0.983588
max        0.999912
dtype: float64
预测偏差量与真实销量百分比的统计指标：
count    570.000000
mean      15.556903
std       22.141838
min        0.206612
25%        3.267049
50%        7.323854
75%       16.063256
max       99.991186
dtype: float64
----------------------------------------
补偿后剩余偏差量的统计指标：
count    609.000000
mean       0.487863
std        0.290290
min        0.017702
25%        0.253823
50%        0.436165
75%        0.745113
max        0.992873
dtype: float64
补偿后剩余偏差量与真实销量百分比的统计指标：
count    570.000000
mean       9.207049
std       15.886382
min        0.021854
25%        1.512303
50%        3.655175
75%        9.221841
max       95.637593
dtype: float64
----------------------------------------
补偿量的统计指标：
count    609.000000
mean       0.337069
std        0.257588
min        0.002265
25%        0.119720
50%        0.280825
7

In [12]:
# 1.3 预测偏低补偿偏高，补偿对侧变好：
print('预测偏低补偿偏高，补偿对侧变好时：\n')

pva_s1_c3 = abs(y_s1_c3 - yhat_s1_c3)  # 预测偏差量
pva_s1_c3_sts = pva_s1_c3.describe()
pvr_s1_c3 = abs(pva_s1_c3[y_s1_c3!=0] / y_s1_c3[y_s1_c3!=0])*100  # 预测偏差量与真实销量百分比
pvr_s1_c3_sts = pvr_s1_c3.describe()
print('预测偏差量的统计指标：')
print(pva_s1_c3_sts)
print('预测偏差量与真实销量百分比的统计指标：')
print(pvr_s1_c3_sts)
print('----------------------------------------')

cva_s1_c3 = abs(y_s1_c3 - ycomp_s1_c3)  # 补偿后剩余偏差量
cva_s1_c3_sts = cva_s1_c3.describe()
cvr_s1_c3 = abs(cva_s1_c3[y_s1_c3!=0] / y_s1_c3[y_s1_c3!=0])*100  # 补偿后剩余偏差量与真实销量百分比
cvr_s1_c3_sts = cvr_s1_c3.describe()
print('补偿后剩余偏差量的统计指标：')
print(cva_s1_c3_sts)
print('补偿后剩余偏差量与真实销量百分比的统计指标：')
print(cvr_s1_c3_sts)
print('----------------------------------------')

ca_s1_c3 = abs(yhat_s1_c3 - ycomp_s1_c3)  # 补偿量
ca_s1_c3_sts = ca_s1_c3.describe()
car_s1_c3 = abs(ca_s1_c3[pva_s1_c3!=0] / pva_s1_c3[pva_s1_c3!=0])*100  # 补偿量占预测偏差量百分比
car_s1_c3_sts = car_s1_c3.describe()
print('补偿量的统计指标：')
print(ca_s1_c3_sts)
print('补偿量占预测偏差量百分比的统计指标：')
print(car_s1_c3_sts)

预测偏低补偿偏高，补偿对侧变好时：

预测偏差量的统计指标：
count    616.000000
mean       0.819288
std        0.215354
min        0.158623
25%        0.748036
50%        0.916522
75%        0.988032
max        0.999990
dtype: float64
预测偏差量与真实销量百分比的统计指标：
count    578.000000
mean      15.223140
std       22.267666
min        0.517869
25%        3.785265
50%        6.813710
75%       14.798720
max       99.999021
dtype: float64
----------------------------------------
补偿后剩余偏差量的统计指标：
count    616.000000
mean       0.487868
std        0.297451
min        0.017702
25%        0.253823
50%        0.436165
75%        0.762558
max        0.990607
dtype: float64
补偿后剩余偏差量与真实销量百分比的统计指标：
count    578.000000
mean       8.944414
std       16.025051
min        0.052064
25%        1.507785
50%        3.678234
75%        8.117245
max       99.060736
dtype: float64
----------------------------------------
补偿量的统计指标：
count    616.000000
mean       1.307156
std        0.447645
min        0.193778
25%        0.993413
50%        1.318239

In [13]:
# 1.4 预测偏高补偿偏低，补偿对侧变好：
print('预测偏高补偿偏低，补偿对侧变好时：\n')

pva_s1_c4 = abs(y_s1_c4 - yhat_s1_c4)  # 预测偏差量
pva_s1_c4_sts = pva_s1_c4.describe()
pvr_s1_c4 = abs(pva_s1_c4[y_s1_c4!=0] / y_s1_c4[y_s1_c4!=0])*100  # 预测偏差量与真实销量百分比
pvr_s1_c4_sts = pvr_s1_c4.describe()
print('预测偏差量的统计指标：')
print(pva_s1_c4_sts)
print('预测偏差量与真实销量百分比的统计指标：')
print(pvr_s1_c4_sts)
print('----------------------------------------')

cva_s1_c4 = abs(y_s1_c4 - ycomp_s1_c4)  # 补偿后剩余偏差量
cva_s1_c4_sts = cva_s1_c4.describe()
cvr_s1_c4 = abs(cva_s1_c4[y_s1_c4!=0] / y_s1_c4[y_s1_c4!=0])*100  # 补偿后剩余偏差量与真实销量百分比
cvr_s1_c4_sts = cvr_s1_c4.describe()
print('补偿后剩余偏差量的统计指标：')
print(cva_s1_c4_sts)
print('补偿后剩余偏差量与真实销量百分比的统计指标：')
print(cvr_s1_c4_sts)
print('----------------------------------------')

ca_s1_c4 = abs(yhat_s1_c4 - ycomp_s1_c4)  # 补偿量
ca_s1_c4_sts = ca_s1_c4.describe()
car_s1_c4 = abs(ca_s1_c4[pva_s1_c4!=0] / pva_s1_c4[pva_s1_c4!=0])*100  # 补偿量占预测偏差量百分比
car_s1_c4_sts = car_s1_c4.describe()
print('补偿量的统计指标：')
print(ca_s1_c4_sts)
print('补偿量占预测偏差量百分比的统计指标：')
print(car_s1_c4_sts)

预测偏高补偿偏低，补偿对侧变好时：

预测偏差量的统计指标：
count    599.000000
mean       0.820166
std        0.208742
min        0.123573
25%        0.733190
50%        0.901788
75%        0.983588
max        0.999912
dtype: float64
预测偏差量与真实销量百分比的统计指标：
count    561.000000
mean      13.983127
std       20.212582
min        0.415059
25%        3.183690
50%        6.191296
75%       14.284455
max       99.991186
dtype: float64
----------------------------------------
补偿后剩余偏差量的统计指标：
count    599.000000
mean       0.473872
std        0.286633
min        0.008851
25%        0.245252
50%        0.444113
75%        0.739181
max        0.985146
dtype: float64
补偿后剩余偏差量与真实销量百分比的统计指标：
count    561.000000
mean       7.980950
std       13.921148
min        0.009945
25%        1.330450
50%        3.129114
75%        7.136378
max       96.611777
dtype: float64
----------------------------------------
补偿量的统计指标：
count    599.000000
mean       1.294039
std        0.429881
min        0.149971
25%        1.020118
50%        1.337480

In [26]:
# 1.5 预测偏低，完全补偿：
print('预测偏低，完全补偿时：\n')

pva_s1_c5 = abs(y_s1_c5 - yhat_s1_c5)  # 预测偏差量
pva_s1_c5_sts = pva_s1_c5.describe()
pvr_s1_c5 = abs(pva_s1_c5[y_s1_c5!=0] / y_s1_c5[y_s1_c5!=0])*100  # 预测偏差量与真实销量百分比
pvr_s1_c5_sts = pvr_s1_c5.describe()
print('预测偏差量的统计指标：')
print(pva_s1_c5_sts)
print('预测偏差量与真实销量百分比的统计指标：')
print(pvr_s1_c5_sts)
print('----------------------------------------')

cva_s1_c5 = abs(y_s1_c5 - ycomp_s1_c5)  # 补偿后剩余偏差量
cva_s1_c5_sts = cva_s1_c5.describe()
cvr_s1_c5 = abs(cva_s1_c5[y_s1_c5!=0] / y_s1_c5[y_s1_c5!=0])*100  # 补偿后剩余偏差量与真实销量百分比
cvr_s1_c5_sts = cvr_s1_c5.describe()
if ((cva_s1_c5==0).sum() == len(cva_s1_c5)) & ((cvr_s1_c5==0).sum() == len(cvr_s1_c5)):
    print('补偿后剩余偏差量的统计指标：')
    print(cva_s1_c5_sts)
    print('补偿后剩余偏差量与真实销量百分比的统计指标：')
    print(cvr_s1_c5_sts)
    print('----------------------------------------')
else: raise Exception('1.5 在第一大类的第五小类中，即预测偏低，完全补偿这种情况下，\
补偿偏差量(cva_s1_c5)和/或补偿偏差比(cvr_s1_c5)存在计算有误的情况，请检查错误原因。')

ca_s1_c5 = abs(yhat_s1_c5 - ycomp_s1_c5)  # 补偿量
ca_s1_c5_sts = ca_s1_c5.describe()
car_s1_c5 = abs(ca_s1_c5[pva_s1_c5!=0] / pva_s1_c5[pva_s1_c5!=0])*100  # 补偿量占预测偏差量百分比
car_s1_c5_sts = car_s1_c5.describe()
if ((ca_s1_c5==pva_s1_c5).sum() == len(ca_s1_c5)) & ((car_s1_c5==100).sum() == len(car_s1_c5)):
    print('补偿量的统计指标：')
    print(ca_s1_c5_sts)
    print('补偿量占预测偏差量百分比的统计指标：')
    print(car_s1_c5_sts)
else: raise Exception('1.5 在第一大类的第五小类中，即预测偏低，完全补偿这种情况下，\
补偿量(ca_s1_c5)、补偿比(car_s1_c5)、预测偏差量(pva_s1_c5)中存在计算有误的情况，请检查错误原因。')

预测偏低，完全补偿时：

预测偏差量的统计指标：
count    26.000000
mean      0.660191
std       0.281095
min       0.114785
25%       0.414056
50%       0.713880
75%       0.911873
max       0.993889
dtype: float64
预测偏差量与真实销量百分比的统计指标：
count    24.000000
mean     12.768981
std      13.816208
min       1.203198
25%       4.594829
50%       6.928789
75%      16.580220
max      52.155100
dtype: float64
----------------------------------------
补偿后剩余偏差量的统计指标：
count    26.0
mean      0.0
std       0.0
min       0.0
25%       0.0
50%       0.0
75%       0.0
max       0.0
dtype: float64
补偿后剩余偏差量与真实销量百分比的统计指标：
count    24.0
mean      0.0
std       0.0
min       0.0
25%       0.0
50%       0.0
75%       0.0
max       0.0
dtype: float64
----------------------------------------
补偿量的统计指标：
count    26.000000
mean      0.660191
std       0.281095
min       0.114785
25%       0.414056
50%       0.713880
75%       0.911873
max       0.993889
dtype: float64
补偿量占预测偏差量百分比的统计指标：
count     26.0
mean     100.0
std        0.0
min   

In [27]:
# 1.6 预测偏高，完全补偿：
print('预测偏高，完全补偿时：\n')

pva_s1_c6 = abs(y_s1_c6 - yhat_s1_c6)  # 预测偏差量
pva_s1_c6_sts = pva_s1_c6.describe()
pvr_s1_c6 = abs(pva_s1_c6[y_s1_c6!=0] / y_s1_c6[y_s1_c6!=0])*100  # 预测偏差量与真实销量百分比
pvr_s1_c6_sts = pvr_s1_c6.describe()
print('预测偏差量的统计指标：')
print(pva_s1_c6_sts)
print('预测偏差量与真实销量百分比的统计指标：')
print(pvr_s1_c6_sts)
print('----------------------------------------')

cva_s1_c6 = abs(y_s1_c6 - ycomp_s1_c6)  # 补偿后剩余偏差量
cva_s1_c6_sts = cva_s1_c6.describe()
cvr_s1_c6 = abs(cva_s1_c6[y_s1_c6!=0] / y_s1_c6[y_s1_c6!=0])*100  # 补偿后剩余偏差量与真实销量百分比
cvr_s1_c6_sts = cvr_s1_c6.describe()
if ((cva_s1_c6==0).sum() == len(cva_s1_c6)) & ((cvr_s1_c6==0).sum() == len(cvr_s1_c6)):
    print('补偿后剩余偏差量的统计指标：')
    print(cva_s1_c6_sts)
    print('补偿后剩余偏差量与真实销量百分比的统计指标：')
    print(cvr_s1_c6_sts)
    print('----------------------------------------')
else: raise Exception('1.6 在第一大类的第六小类中，即预测偏高，完全补偿这种情况下，\
补偿偏差量(cva_s1_c6)和/或补偿偏差比(cvr_s1_c6)存在计算有误的情况，请检查错误原因。')

ca_s1_c6 = abs(yhat_s1_c6 - ycomp_s1_c6)  # 补偿量
ca_s1_c6_sts = ca_s1_c6.describe()
car_s1_c6 = abs(ca_s1_c6[pva_s1_c6!=0] / pva_s1_c6[pva_s1_c6!=0])*100  # 补偿量占预测偏差量百分比
car_s1_c6_sts = car_s1_c6.describe()
if ((ca_s1_c6==pva_s1_c6).sum() == len(ca_s1_c6)) & ((car_s1_c6==100).sum() == len(car_s1_c6)):
    print('补偿量的统计指标：')
    print(ca_s1_c6_sts)
    print('补偿量占预测偏差量百分比的统计指标：')
    print(car_s1_c6_sts)
else: raise Exception('1.6 在第一大类的第六小类中，即预测偏高，完全补偿这种情况下，\
补偿量(ca_s1_c6)、补偿比(car_s1_c6)、预测偏差量(pva_s1_c6)中存在计算有误的情况，请检查错误原因。')

预测偏高，完全补偿时：

预测偏差量的统计指标：
count    23.000000
mean      0.632434
std       0.361065
min       0.017702
25%       0.273482
50%       0.733190
75%       0.966080
max       0.999520
dtype: float64
预测偏差量与真实销量百分比的统计指标：
count    22.000000
mean     18.504726
std      23.634508
min       0.057103
25%       3.547301
50%       8.693388
75%      30.319031
max      98.935825
dtype: float64
----------------------------------------
补偿后剩余偏差量的统计指标：
count    23.0
mean      0.0
std       0.0
min       0.0
25%       0.0
50%       0.0
75%       0.0
max       0.0
dtype: float64
补偿后剩余偏差量与真实销量百分比的统计指标：
count    22.0
mean      0.0
std       0.0
min       0.0
25%       0.0
50%       0.0
75%       0.0
max       0.0
dtype: float64
----------------------------------------
补偿量的统计指标：
count    23.000000
mean      0.632434
std       0.361065
min       0.017702
25%       0.273482
50%       0.733190
75%       0.966080
max       0.999520
dtype: float64
补偿量占预测偏差量百分比的统计指标：
count     23.0
mean     100.0
std        0.0
min   

In [72]:
# 2. 对于第二大类即补偿变差的情况

m21, m22 = len(y_s2_c1), len(y_s2_c2)  # 同侧补偿且变差
m23, m24 = len(y_s2_c3), len(y_s2_c4)  # 对侧补偿且变差
m25, m26 = len(y_s2_c5), len(y_s2_c6)  # 完全准确预测
m2syn, m2opp, m2com = m21+m22, m23+m14, m25+m26  # 补偿变差时同侧、对侧、完全准确预测的单品数
if m21+m22+m23+m24+m25+m26 == m2:
    print('计算正确，当补偿变差时，其下6种子情况的单品总数相加与补偿变差的单品总数相等：\n')
    print('预测销量与补偿后销量均比真实销量低，补偿变差且发生在同侧，该种情况有：{:.0f}个单品，\
占补偿变差单品数的比例为：{:.2f}%'.format(m21, m21/m2*100))
    print('预测销量与补偿后销量均比真实销量高，补偿变差且发生在同侧，该种情况有：{:.0f}个单品，\
占补偿变差单品数的比例为：{:.2f}%'.format(m22, m22/m2*100))
    print('预测销量偏低，补偿后销量偏高，补偿变差且发生在对侧，该种情况有：{:.0f}个单品，\
占补偿变差单品数的比例为：{:.2f}%'.format(m23, m23/m2*100))
    print('预测销量偏高，补偿后销量偏低，补偿变差且发生在对侧，该种情况有：{:.0f}个单品，\
占补偿变差单品数的比例为：{:.2f}%'.format(m24, m24/m2*100))
    print('完全准确预测，补偿后销量偏低，该种情况有：{:.0f}个单品，占补偿变差单品数的比例为：{:.2f}%'.format(m25, m25/m2*100))
    print('完全准确预测，补偿后销量偏高，该种情况有：{:.0f}个单品，占补偿变差单品数的比例为：{:.2f}%'.format(m26, m26/m2*100))
    print('补偿变差且发生在同侧的单品数有：{:.0f}个，占补偿变差单品数的比例为：{:.2f}%'.format(m2syn, m2syn/m2*100))
    print('补偿变差且发生在对侧的单品数有：{:.0f}个，占补偿变差单品数的比例为：{:.2f}%'.format(m2opp, m2opp/m2*100))
    print('完全准确预测的单品数有：{:.0f}个，占补偿变差单品数的比例为：{:.2f}%'.format(m2com, m2com/m2*100))  
else: raise Exception('计算有误，当补偿变差时，其下6种子情况的单品总数相加与补偿变差的单品总数不等。')

计算正确，当补偿变差时，其下6种子情况的单品总数相加与补偿变差的单品总数相等：

预测销量与补偿后销量均比真实销量低，补偿变差且发生在同侧，该种情况有：599个单品，占补偿变差单品数的比例为：24.19%
预测销量与补偿后销量均比真实销量高，补偿变差且发生在同侧，该种情况有：586个单品，占补偿变差单品数的比例为：23.67%
预测销量偏低，补偿后销量偏高，补偿变差且发生在对侧，该种情况有：606个单品，占补偿变差单品数的比例为：24.47%
预测销量偏高，补偿后销量偏低，补偿变差且发生在对侧，该种情况有：638个单品，占补偿变差单品数的比例为：25.77%
完全准确预测，补偿后销量偏低，该种情况有：17个单品，占补偿变差单品数的比例为：0.69%
完全准确预测，补偿后销量偏高，该种情况有：30个单品，占补偿变差单品数的比例为：1.21%
补偿变差且发生在同侧的单品数有：1185个，占补偿变差单品数的比例为：47.86%
补偿变差且发生在对侧的单品数有：1205个，占补偿变差单品数的比例为：48.67%
完全准确预测的单品数有：47个，占补偿变差单品数的比例为：1.90%


In [69]:
# 2.1 预测补偿均低，补偿同侧变差：
print('预测补偿均低，补偿同侧变差时：\n')

pva_s2_c1 = abs(y_s2_c1 - yhat_s2_c1)  # 预测偏差量
pva_s2_c1_sts = pva_s2_c1.describe()
pvr_s2_c1 = abs(pva_s2_c1[y_s2_c1!=0] / y_s2_c1[y_s2_c1!=0])*100  # 预测偏差量与真实销量百分比
pvr_s2_c1_sts = pvr_s2_c1.describe()
print('预测偏差量的统计指标：')
print(pva_s2_c1_sts)
print('预测偏差量与真实销量百分比的统计指标：')
print(pvr_s2_c1_sts)
print('----------------------------------------')

cva_s2_c1 = abs(y_s2_c1 - ycomp_s2_c1)  # 补偿后剩余偏差量
cva_s2_c1_sts = cva_s2_c1.describe()
cvr_s2_c1 = abs(cva_s2_c1[y_s2_c1!=0] / y_s2_c1[y_s2_c1!=0])*100  # 补偿后剩余偏差量与真实销量百分比
cvr_s2_c1_sts = cvr_s2_c1.describe()
print('补偿后剩余偏差量的统计指标：')
print(cva_s2_c1_sts)
print('补偿后剩余偏差量与真实销量百分比的统计指标：')
print(cvr_s2_c1_sts)
print('----------------------------------------')

ca_s2_c1 = abs(yhat_s2_c1 - ycomp_s2_c1)  # 补偿量
ca_s2_c1_sts = ca_s2_c1.describe()
car_s2_c1 = abs(ca_s2_c1[pva_s2_c1!=0] / pva_s2_c1[pva_s2_c1!=0])*100  # 补偿量占预测偏差量百分比
car_s2_c1_sts = car_s2_c1.describe()
print('补偿量的统计指标：')
print(ca_s2_c1_sts)
print('补偿量占预测偏差量百分比的统计指标：')
print(car_s2_c1_sts)
print('----------------------------------------')

if (len(pva_s2_c1)==len(cva_s2_c1)==len(ca_s2_c1)==len(car_s2_c1)==m21) & (len(pva_s2_c1)>=len(pvr_s2_c1)) & (len(cva_s2_c1)>=len(cvr_s2_c1)): # 按位与运算符(&)的优先级高于比较运算符和等于运算符，所以需用()括起每个条件
    print('在2.1预测补偿均低，补偿同侧变差的情况中，各分量的长度匹配正确。')
else: raise Exception('在2.1即预测补偿均低，补偿同侧变差的情况中，各分量的长度匹配存在错误，请检查该异常。')


预测补偿均低，补偿同侧变差时：

预测偏差量的统计指标：
count    599.000000
mean       0.475199
std        0.296214
min        0.008851
25%        0.245252
50%        0.521551
75%        0.750987
max        0.999207
dtype: float64
预测偏差量与真实销量百分比的统计指标：
count    562.000000
mean       8.070667
std       13.489879
min        0.015002
25%        1.253759
50%        3.054119
75%        8.216913
max       90.557836
dtype: float64
----------------------------------------
补偿后剩余偏差量的统计指标：
count    599.000000
mean       0.831669
std        0.205053
min        0.026551
25%        0.756802
50%        0.916522
75%        0.988032
max        0.999990
dtype: float64
补偿后剩余偏差量与真实销量百分比的统计指标：
count    562.000000
mean      14.331190
std       21.076070
min        0.221260
25%        3.198978
50%        6.165794
75%       15.695151
max       99.999021
dtype: float64
----------------------------------------
补偿量的统计指标：
count    599.000000
mean       0.356470
std        0.272696
min        0.000783
25%        0.121099
50%        0.297764
7